# Collateral Repo Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-repo-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [60]:
import refinitiv.dataplatform as rdp
import pandas as pd

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Repo interest & repurchase price

This sample computes the repurchase price and the interest due for a collateral repo. In this example the repo:
- repo starts on 2021-01-21
- repo ends on 2021-01-22
- interest paid is 3.03%
- the daycouont basis is Dcb_Actual_360
- the underlying bonds is a Indonesian government bonds (FR0077)
- the bpnd notional is IDR300,000,000,000
- the bond dirty price is 109.330619
- a 10% haircut is applied

in addition the the parameters above the following 'fields' are requested:
- the InitialMarginPercent
- the RepoInterestAmountInDealCcy
- the RepurchasePrice that is computed from the purchase price and the repo interest
- UnderlyingAtStartObject that represents the state at the beginning of the repo of the bond used as collateral
- the error information (ErrorCode,ErrorMessage)


In [66]:
request_body = {
    
    "fields" : ["InstrumentTag","StartDate","EndDate",
                "SettlementConvention","DayCountBasis"
                "RepoRatePercent","HaircutRatePercent","InitialMarginPercent",
                "PurchasePrice","RepurchasePrice","RepoInterestAmountInDealCcy",
                "UnderlyingAtStartObject","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Repo",
            "instrumentDefinition": {
                "startDate":"2021-01-21",
                "endDate":"2021-01-22",
                "repoRatePercent":3.03,
                "dayCountBasis":"Dcb_Actual_360",
                "isCouponExchanged":False,
                "underlyingInstruments":[{
                    "instrumentType": "Bond",
                    "instrumentDefinition":{
                        "instrumentCode":"FR0077",
                        "NotionalAmount":300000000000,
                    },
                    "pricingParameters":{
                        "RepoParameters":{
                            "haircutRatePercent":10
                        },
                        "PricingParametersAtStart":
                        {
                            "dirtyPrice":109.330619
                        }
                    }
                 }]
             }
        }],
    
    "pricingParameters": {
        "valuationDate": "2021-01-21"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,StartDate,EndDate,SettlementConvention,DayCountBasisRepoRatePercent,HaircutRatePercent,InitialMarginPercent,PurchasePrice,RepurchasePrice,RepoInterestAmountInDealCcy,UnderlyingAtStartObject,ErrorCode,ErrorMessage
0,None,2021-01-21T00:00:00Z,2021-01-22T00:00:00Z,2WD,None,10.0,111.111111,2.951927e+11,2.952175e+11,2.484538e+07,"{'endDate': {'value': '2024-05-15T00:00:00Z', ...",,


### displaying the state of the bond posted as collateral at the start of the repo

In [65]:
pd.DataFrame(df["UnderlyingAtStartObject"][0])

,endDate,accrued,cleanPrice,dirtyPrice,notionalAmount,accruedDays,couponRatePercent,marketValueInDealCcy,yieldPercent
value,2024-05-15T00:00:00Z,1.5038,107.827,109.331,3e+11,67,8.125,3.27992e+11,5.5054
status,Data,Computed,Computed,User,User,Computed,Data,Computed,Computed
